In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Analyza datasetu

In [2]:
survival_data = pd.read_csv('train.csv')
survival_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
survival_data.shape

(891, 12)

In [4]:
survival_data=survival_data.drop(columns =['Cabin'])
survival_data = survival_data.dropna()
survival_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,Q
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


In [5]:
survival_data = survival_data[['Sex', 'Age','Pclass','Survived']]
X = survival_data.drop(columns=['Survived'])
y = survival_data['Survived']

# Změnění pohlaví na 0 a 1

In [15]:
X['Sex'].replace('female', 0,inplace=True)
X['Sex'].replace('male', 1,inplace=True)

,Sex,Age,Pclass
0,1,34,3
1,0,47,3
2,1,62,2
3,1,27,3
4,0,22,3
...,...,...,...
409,0,3,3
411,0,37,1
412,0,28,3
414,0,39,1


In [7]:
accuracy = 0
bestprediction = None
bestmodel = None

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2,random_state= 135)
X['Age'] = X['Age'].astype('int64')

# Trénuje model měněním criterion

In [9]:
for crit in ['entropy','gini']:
    model = DecisionTreeClassifier(criterion= crit)
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    score=accuracy_score(y_valid, predictions)
    if accuracy < score:
        accuracy = score
        bestprediction = predictions
        bestmodel = model
    print(score)

0.8041958041958042
0.8041958041958042


# Trénuje model měněním Max_depth

In [10]:
# Cyklus měnící hloubku DecisionTreeClassifier
for depth in [1,8,16,32,None]:
    model = DecisionTreeClassifier(max_depth=depth)
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    score=accuracy_score(y_valid, predictions)
    # Uložení nelepšího modelu
    if score > accuracy:
        accuracy = score
        bestprediction = predictions
        bestmodel = model
    print(score)       

0.7832167832167832
0.7972027972027972
0.8041958041958042
0.8041958041958042
0.8041958041958042


# Trénuje model na RandomForestClassifier(n_estimators, max_samples, random_state)

In [11]:
for estimators in [10,100,1000,10000]:
    for samples in [5,10,50,569]:
        model = RandomForestClassifier(n_estimators = estimators, max_samples = samples,random_state = 2)
        model.fit(X_train, y_train)
        predictions = model.predict(X_valid)
        score=accuracy_score(y_valid, predictions)
        if score > accuracy:
            accuracy = score
            bestprediction = predictions
            bestmodel = model
        print(score)

0.7132867132867133
0.7692307692307693
0.7972027972027972
0.8111888111888111
0.7272727272727273
0.7832167832167832
0.7972027972027972
0.8251748251748252
0.7622377622377622
0.7762237762237763
0.7972027972027972
0.8251748251748252
0.7622377622377622
0.7832167832167832
0.8041958041958042
0.8181818181818182


In [12]:
accuracy

0.8251748251748252

In [13]:
bestmodel

RandomForestClassifier(max_samples=569, random_state=2)

# Aplikování nejlepšího modelu do testovacíh dat

In [14]:
survival_data = pd.read_csv('test.csv')
survival_data=survival_data.drop(columns =['Cabin'])
survival_data = survival_data[['Sex', 'Age','Pclass']]
survival_data = survival_data.dropna()
X = survival_data
X['Sex'].replace('female', 0,inplace=True)
X['Sex'].replace('male', 1,inplace=True)
X['Age'] = X['Age'].astype('int64')
predictions = bestmodel.predict(X)
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,